In [17]:
import numpy as np
import pandas as pd
import os
from xgboost import XGBRegressor as XGBR
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LinearRegression
from sklearn import datasets
from sklearn.model_selection import KFold,cross_val_score,train_test_split
from sklearn.metrics import mean_squared_error
import matplotlib.pyplot as plt
from time import time
import datetime
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score, mean_squared_error
from sklearn.preprocessing import StandardScaler

In [18]:
path = 'train.csv'
path1 = 'unique_m.csv'
Dataset1 = pd.read_csv(path)
Dataset2 = pd.read_csv(path1).drop(['critical_temp', 'material'], axis = 1)

dataFrame = pd.concat([Dataset1, Dataset2], axis = 1)

In [19]:
# 删除一切相关联的特征,进行抽象
correlationMatrix = dataFrame.corr()
correlationFeatures = set()

for i in range(len(correlationMatrix)) :
    for j in range(i) :
        if abs(correlationMatrix.iloc[i, j]) > 0.75 :
            colname = correlationMatrix.columns[i]
            if colname != "critical_temp" :
                correlationFeatures.add(colname)
dataFrame.drop(correlationFeatures, axis = 1, inplace = True)
dataFrame.head(5)

,number_of_elements,mean_atomic_mass,range_atomic_mass,wtd_range_atomic_mass,mean_fie,wtd_mean_fie,wtd_range_fie,mean_atomic_radius,wtd_range_atomic_radius,range_Density,...,Ir,Pt,Au,Hg,Tl,Pb,Bi,Po,At,Rn
0,4,88.944468,122.90607,31.794921,775.425,1010.268571,735.985714,160.25,42.914286,8958.571,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0
1,5,92.729214,122.90607,36.161939,766.440,1010.612857,743.164286,161.20,50.571429,10488.571,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0
2,4,88.944468,122.90607,35.741099,775.425,1010.820000,743.164286,160.25,49.314286,8958.571,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0
3,4,88.944468,122.90607,33.768010,775.425,1010.544286,739.575000,160.25,46.114286,8958.571,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0
4,4,88.944468,122.90607,27.848743,775.425,1009.717143,728.807143,160.25,36.514286,8958.571,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0


In [20]:
# 把方差低的特征剔除
from sklearn.feature_selection import VarianceThreshold

selector = VarianceThreshold(threshold = 0.1)
selector.fit(dataFrame)
dataFrame = pd.DataFrame(selector.transform(dataFrame), columns = dataFrame.columns[selector.get_support()])
dataFrame.shape

(21263, 63)

In [21]:
dataFrame.to_csv('CleanData.csv')

In [22]:
sc = StandardScaler()

df = pd.read_csv('CleanData.csv')
x = df.drop('critical_temp', axis = 1)
x_scale = sc.fit_transform(x)
x = pd.DataFrame(x_scale, columns = x.columns)
y = df['critical_temp']

x_train, x_test, y_train, y_test = train_test_split(x, y, test_size = 0.25, random_state = 42)

In [35]:
reg = XGBR(n_estimators=988,
           max_depth = 8,
           eta = 0.11478691728033347,
          ).fit(x_train,y_train)
y_prediction = pd.Series(reg.predict(x_test))
min_rmse = round(np.sqrt(mean_squared_error(y_test, y_prediction)), 4)
r2 = round(r2_score(y_prediction, y_test), 4)

print(r2)
print(min_rmse)

0.9353
8.4193


In [36]:
y_test.to_csv('y_test2.csv')

In [37]:
y_prediction.to_csv('y_predict2.csv')